# MNIST Syft Data Owner

In [1]:
import syft as sy
sy.LOG_FILE = "syft_do.log"
_ = sy.logger.add(sy.LOG_FILE, enqueue=True, colorize=False, diagnose=True, backtrace=True, level="TRACE")

# PART 1: Launch a Duet Server and Connect

As a data owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server (within this notebook).
We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a data scientist) connect to this server.

You do this by running the code below and following the instructions it gives!

In [2]:
# duet = sy.launch_duet(loopback=True)
duet = sy.launch_duet(network_url="http://localhost:5000", loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at: http://localhost:5000
♫♫♫ > http://localhost:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... id {
  value: "\"\274\326\326i~@\340\236\262\003\010H7\032\265"
}
name: "om-net"
node {
  id {
    value: "\"\274\326\326i~@\340\236\262\003\010H7\032\265"
  }
  name: "om-net"
}

DONE!
♫♫♫ >
♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet('e71c3a657a9f8f0463b88bd9ee284b87')

♫♫♫ > STEP 2: The code above will print out a 'Client Id'. Have
♫♫♫ >         your duet partner send it to you and enter it below!

Running loopback mode. Use sy.join_duet(loopback=True) on the other side.
♫♫♫ > Connecting...
♫♫♫ > ...using a running event loop...


2020-10-31 18:55:59.103 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 18:56:04.108 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0

♫♫♫ > CONNECTED!

♫♫♫ > DUET LIVE STATUS  -  Objects: 3  Requests: 1   Messages: 23                                                         > Request cuda_is_available: To run test and inference locally
♫♫♫ > DUET LIVE STATUS  *  Objects: 13  Requests: 0   Messages: 137                                                        Launcher PQ: 0 / 0 - CQ: 0 / 0
♫♫♫ > DUET LIVE STATUS  -  Objects: 28  Requests: 0   Messages: 426                                Launcher PQ: 0 / 0 - CQ: 0 / 0
♫♫♫ > DUET LIVE STATUS  -  Objects: 28  Requests: 0   Messages: 1480                                Launcher PQ: 0 / 0 - CQ: 0 / 0
♫♫♫ > DUET LIVE STATUS  -  Objects: 27  Requests: 0   Messages: 2484                                Launcher PQ: 0 / 0 - CQ: 0 / 0
♫♫♫ > DUET LIVE STATUS  -  Objects: 26  Requests: 0   Messages: 4115                                Launcher PQ: 0 / 0 - CQ: 0 / 0
♫♫♫ > DUET LIVE STATUS  -  Objects: 28  Requests: 0   Messages: 6775                    

In [3]:
# accept all incoming requests, print and log them
accept_handler = {
    "request_name": "age_data",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "cuda_is_available",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
# accept_handler = {
#     "request_name": "loss",
#     "timeout_secs": -1,
#     "action": "deny",
#     "print_local": True,
#     "log_local": True
# }
# duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "loss",
    "timeout_secs": -1,
    "action": "accept"
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "train_size",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "inference",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "model_download",
    "timeout_secs": -1,
    "action": "accept",
}
duet.requests.add_handler(accept_handler)

2020-10-31 18:56:09.114 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


In [4]:
duet.requests.handlers

[{'request_name': 'age_data',
  'timeout_secs': -1,
  'action': 'accept',
  'print_local': True,
  'log_local': True,
  'created_time': 1604134528.0},
 {'request_name': 'cuda_is_available',
  'timeout_secs': -1,
  'action': 'accept',
  'print_local': True,
  'log_local': True,
  'created_time': 1604134528.0},
 {'request_name': 'loss',
  'timeout_secs': -1,
  'action': 'accept',
  'created_time': 1604134528.0},
 {'request_name': 'train_size',
  'timeout_secs': -1,
  'action': 'accept',
  'print_local': True,
  'log_local': True,
  'created_time': 1604134528.0},
 {'request_name': 'inference',
  'timeout_secs': -1,
  'action': 'accept',
  'print_local': True,
  'log_local': True,
  'created_time': 1604134528.0},
 {'request_name': 'model_download',
  'timeout_secs': -1,
  'action': 'accept',
  'created_time': 1604134528.0}]

2020-10-31 18:56:14.118 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


In [ ]:
for handler in duet.requests.handlers:
    del handler["created_time"]
    duet.requests.remove_handler(handler)

In [ ]:
# deny all incoming requests, print and log them
deny_handler = {
    "request_name": "loss",
    "timeout_secs": -1,
    "action": "deny",
    "print_local": True,
    "log_local": True
}
duet.requests.remove_handler(deny_handler)

In [ ]:
accept_handler = {
    "request_name": "loss",
    "timeout_secs": -1,
    "action": "accept"
}
duet.requests.add_handler(accept_handler)

In [ ]:
# # accept all incoming requests, print and log them
# accept_handler = {
#     "timeout_secs": -1,
#     "action": "accept",
#     "print_local": True,
#     "log_local": True
# }
# duet.requests.add_handler(accept_handler)

If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# PART 2: Launch a Duet Server and Connect

In [5]:
import torch as th
import torchvision as tv

In [6]:
# # let's pretend we have a tensor of human ages
# age_data = th.rand((9216, 128)) # not working too big??
# import sys
# sys.getsizeof(age_data)

In [7]:
# age_data = th.rand((9216, 128)) # works Wall time: 19.4 s
age_data = th.rand((9216, 96)) # works Wall time: 46.6 s
# age_data = th.rand((4608, 128))   # 589824 params # works Wall time: 24.7 s
# age_data = th.rand((10, 1))   # params # works Wall time: 24.7 s
age_data_pointer = age_data.send(duet, searchable=True)

# Let's tag it as "age data"
age_data = age_data.tag("#age_data")

# Let's add a description to the tensor
age_data = age_data.describe("This is a list of people's ages. Let's keep it private!")

# Let's UPLOAD THE DATA to the Duet server!
age_data_pointer = age_data.send(duet, searchable=True)

2020-10-31 18:56:19.123 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 18:56:23.683 | INFO     | syft.core.node.domain.domain:check_handler:216 - > Request age_data: I want to see the age data
Value: tensor([[0.4980, 0.5522, 0.2799,  ..., 0.5360, 0.8827, 0.5070],
        [0.9677, 0.4784, 0.4550,  ..., 0.7684, 0.0693, 0.1368],
        [0.8279, 0.9302, 0.7880,  ..., 0.6352, 0.8049, 0.6571],
        ...,
        [0.1649, 0.4891, 0.6087,  ..., 0.8672, 0.2302, 0.2730],
        [0.5153, 0.0215, 0.5692,  ..., 0.1501, 0.1913, 0.1747],
        [0.6480, 0.4488, 0.1111,  ..., 0.0328, 0.5419, 0.3349]])


> Request age_data: I want to see the age data
Value: tensor([[0.4980, 0.5522, 0.2799,  ..., 0.5360, 0.8827, 0.5070],
        [0.9677, 0.4784, 0.4550,  ..., 0.7684, 0.0693, 0.1368],
        [0.8279, 0.9302, 0.7880,  ..., 0.6352, 0.8049, 0.6571],
        ...,
        [0.1649, 0.4891, 0.6087,  ..., 0.8672, 0.2302, 0.2730],
        [0.5153, 0.0215, 0.5692,  ..., 0.1501, 0.1913, 0.1747],
        [0.6480, 0.4488, 0.1111,  ..., 0.0328, 0.5419, 0.3349]])


2020-10-31 18:56:24.126 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


...

Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 18:58:19.312 | CRITICAL | syft.core.node.common.action.garbage_collect_object_action:execute_action:32 - > Unable to delete id_at_location=<UID:268e5764-3a48-4023-824d-500f46231a3c>
2020-10-31 18:58:24.246 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0
2020-10-31 18:58:29.253 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 18:58:34.256 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


In [ ]:
duet.requests.remove_handler(accept_handler)

In [ ]:
# accept all incoming requests, print and log them
deny_handler = {
    "timeout_secs": 10,
    "action": "deny",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(deny_handler)

In [ ]:
duet.requests.remove_handler(deny_handler)

In [8]:
duet.requests.pandas

""


2020-10-31 18:58:39.267 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 18:58:44.274 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0
2020-10-31 18:58:49.275 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


...

Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 19:21:27.670 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0
2020-10-31 19:21:32.672 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


2020-10-31 19:21:37.678 | CRITICAL | syft.grid.connections.webrtc:heartbeat:562 - Launcher PQ: 0 / 0 - CQ: 0 / 0


Launcher PQ: 0 / 0 - CQ: 0 / 0


In [ ]:
print(duet.requests)
if len(duet.requests.requests) > 0:
    duet.requests[0].approve()
duet.requests.pandas    